# Working with MongoDB
- You need to install [MongoDB](https://www.mongodb.com/) to use this notebook.

**NOTE**: Running last cell will close database connection.

In [1]:
const MongoClient = require("mongodb").MongoClient; 
const ObjectId = require("mongodb").ObjectId; 

## Preparing DB

In [2]:
const dbName = "nodejs-learners-package-db";
const dbPort = '27017'; // Set your MongoDB port
const uri = `mongodb://127.0.0.1:${dbPort}/${dbName}`; // Replace with your actual connection uri
const client = new MongoClient(uri);

## Defining some helper functions

In [3]:
// Connects to the DB
async function connectToMongoDB() {
    try {
        await client.connect();
        console.log("Connected to MongoDB!");
    } catch (error) {
        console.error("Error connecting to MongoDB:", error);
        throw error; // Rethrow the error to be caught outside
    }
}

/**
 * Inserts a single document into a specified collection.
 * 
 * @param {string} collectionName - The name of the collection to insert into.
 * @param {object} doc - The document to insert.
 * @returns {Promise<object>} - The result of the insertion operation.
 * @throws {Error} - Throws an error if the insertion fails.
 */
async function createDoc(collectionName, doc) {
    const db = client.db(dbName);
    const collection = db.collection(collectionName);

    try {
        const result = await collection.insertOne(doc);
        console.log("Document created successfully:", result.insertedId);
        return result;
    } catch (error) {
        console.error("Error creating document:", error);
        throw error; // Rethrow the error to be caught outside
    }
}

/**
 * Inserts multiple documents into a specified collection.
 * 
 * @param {string} collectionName - The name of the collection to insert into.
 * @param {Array<object>} docs - An array of documents to insert.
 * @returns {Promise<object>} - The result of the insertion operation.
 * @throws {Error} - Throws an error if the insertion fails.
 */
async function createDocs(collectionName, docs) {
    const db = client.db(dbName);
    const collection = db.collection(collectionName);

    try {
        // Insert multiple documents using insertMany
        const result = await collection.insertMany(docs);
        console.log("Documents created successfully:", result.insertedIds);
        return result;
    } catch (error) {
        console.error("Error creating documents:", error);
        throw error; // Rethrow the error to be caught outside
    }
}

/**
 * Finds a single document in a specified collection that matches the filter.
 * 
 * @param {string} collectionName - The name of the collection to query.
 * @param {object} filter - The query filter to find the document.
 * @returns {Promise<object|null>} - The found document or null if no document matches.
 * @throws {Error} - Throws an error if the query fails.
 */
async function findDoc(collectionName, filter) {
    const db = client.db(dbName);
    const collection = db.collection(collectionName);

    try {
        const doc = await collection.findOne(filter);
        if (doc) {
            console.log("Document found:", doc);
        } else {
            console.log("No document found matching the filter.");
        }
        return doc;
    } catch (error) {
        console.error("Error finding document:", error);
        throw error;
    }
}

/**
 * Finds multiple documents in a specified collection that match the filter.
 * 
 * @param {string} collectionName - The name of the collection to query.
 * @param {object} [filter={}] - The query filter to find documents.
 * @param {object} [options={}] - Additional options for the query.
 * @returns {Promise<object[]>} - An array of found documents.
 * @throws {Error} - Throws an error if the query fails.
 */
async function findDocs(collectionName, filter = {}, options = {}) {
    const db = client.db(dbName);
    const collection = db.collection(collectionName);

    try {
        const cursor = collection.find(filter, options);
        const docs = await cursor.toArray(); // Should convert to array
        console.log("Documents found:", docs);
        return docs;
    } catch (error) {
        console.error("Error finding documents:", error);
        throw error;
    }
}

/**
 * Updates a single document that matches the filter in the specified collection.
 * 
 * @param {string} collectionName - The name of the collection to update.
 * @param {object} filter - The query filter to find the document to update.
 * @param {object} update - The update operations to apply.
 * @param {object} [options={}] - Additional options for the update operation.
 * @returns {Promise<object>} - The result of the update operation.
 * @throws {Error} - Throws an error if the update fails.
 */
async function updateDoc(collectionName, filter, update, options = {}) {
    /*
    const options = { 
      upsert: false,   // If true, creates a new document if no document matches the filter
      returnDocument: 'after'  // MongoDB 4.2+ option to return the document after update (used in `findOneAndUpdate`)
    };
    */
    const db = client.db(dbName);
    const collection = db.collection(collectionName);

    try {
        const result = await collection.updateOne(filter, update, options);
        console.log("Document updated successfully:", result);
        return result;
    } catch (error) {
        console.error("Error updating document:", error);
        throw error;
    }
}

/**
 * Deletes a single document that matches the filter in the specified collection.
 * 
 * @param {string} collectionName - The name of the collection to delete from.
 * @param {object} filter - The query filter to find the document to delete.
 * @returns {Promise<object>} - The result of the delete operation.
 * @throws {Error} - Throws an error if the delete operation fails.
 */
async function deleteDoc(collectionName, filter) {
    const db = client.db(dbName);
    const collection = db.collection(collectionName);

    try {
        const result = await collection.deleteOne(filter);
        console.log("Document deleted successfully:", result);
        return result;
    } catch (error) {
        console.error("Error deleting document:", error);
        throw error;
    }
}

/**
 * Deletes multiple documents that match the filter in the specified collection.
 * 
 * @param {string} collectionName - The name of the collection to delete from.
 * @param {object} filter - The query filter to find documents to delete.
 * @returns {Promise<object>} - The result of the delete operation.
 * @throws {Error} - Throws an error if the delete operation fails.
 */
async function deleteDocs(collectionName, filter) {
    const db = client.db(dbName);
    const collection = db.collection(collectionName);

    try {
        const result = await collection.deleteMany(filter);
        console.log("Documents deleted successfully:", result);
        return result;
    } catch (error) {
        console.error("Error deleting documents:", error);
        throw error;
    }
}

/**
 * Counts the number of documents that match the filter in the specified collection.
 * 
 * @param {string} collectionName - The name of the collection to count documents in.
 * @param {object} [filter={}] - The query filter to count documents.
 * @returns {Promise<number>} - The count of documents matching the filter.
 * @throws {Error} - Throws an error if the count operation fails.
 */
async function countDocs(collectionName, filter = {}) {
    const db = client.db(dbName);
    const collection = db.collection(collectionName);

    try {
        const count = await collection.countDocuments(filter);
        console.log("Document count:", count);
        return count;
    } catch (error) {
        console.error("Error counting documents:", error);
        throw error;
    }
}


## Connection to MongoDB

In [4]:
connectToMongoDB()
    .then(() => {
        console.log("Connected to the database.");
    })
    .catch((error) => {
        console.error("Connecting to the database failed", error);
        process.exit(0);
    })

Promise { <pending> }

Connected to MongoDB!
Connected to the database.


## Simple Database operations

In [5]:
// Adds a doc to 'users' collection
createDoc(
    collection = 'users', 
    doc = {
        _id: 1,
        name: 'Robbin',
        age: 37
    })
    .then((res) => {
        console.log(`Doc created: ${JSON.stringify(res)}`);
    })
    .catch((err) => {
        console.log("Doc creation failed.");
    });

updateDoc(
    collection = 'users', 
    filter = {
        _id: 1,
    },
    update = {
        $set: { age: 26 } 
    })
    .then((res) => {
        console.log(`Doc updated: ${JSON.stringify(res)}`);
    })
    .catch((err) => {
        console.log("Doc update failed.");
    });

// Adds some docs to 'users' collection
createDocs(
    collections = 'users', 
    docs = [
        { _id: new ObjectId(), name: 'Robbin', age: 37 }, // With new ObjectId()
        { name: 'Alex', age: 17 } // With auto ObjectId
    ])
    .then((res) => {
        console.log(`Docs created: ${JSON.stringify(res)}`);
    })
    .catch((err) => {
        console.log("Doc creation failed.");
    })

Promise { <pending> }

Document created successfully: 1
Doc created: {"acknowledged":true,"insertedId":1}
Document updated successfully: {
  acknowledged: true,
  modifiedCount: 1,
  upsertedId: null,
  upsertedCount: 0,
  matchedCount: 1
}
Doc updated: {"acknowledged":true,"modifiedCount":1,"upsertedId":null,"upsertedCount":0,"matchedCount":1}
Documents created successfully: {
  '0': new ObjectId('66c97897ff14d6d2443dd297'),
  '1': new ObjectId('66c97897ff14d6d2443dd298')
}
Docs created: {"acknowledged":true,"insertedCount":2,"insertedIds":{"0":"66c97897ff14d6d2443dd297","1":"66c97897ff14d6d2443dd298"}}


In [6]:
// Finds all docs given the filtet
findDocs('users', 
    filter = {
        age: { $gt: 15, $lt: 40 }
    })
    .then(user => console.log("Found user:", user))
    .catch(error => console.error("Error:", error));

// Finds one doc given the filtet
findDoc('users', 
    filter = {
        age: { $gt: 1, $lt: 40 }
    })
    .then(user => console.log("Found user:", user))
    .catch(error => console.error("Error:", error))

Promise { <pending> }

Documents found: [
  {
    _id: new ObjectId('66c978099f2b2a56ee99af94'),
    name: 'Alice',
    age: 25
  },
  {
    _id: new ObjectId('66c97809f27fedd07292508b'),
    name: 'Bob',
    age: 30
  },
  {
    _id: new ObjectId('66c97809f27fedd07292508e'),
    name: 'Dave',
    age: 35
  },
  { _id: 1, name: 'Robbin', age: 26 },
  {
    _id: new ObjectId('66c97897ff14d6d2443dd297'),
    name: 'Robbin',
    age: 37
  },
  {
    _id: new ObjectId('66c97897ff14d6d2443dd298'),
    name: 'Alex',
    age: 17
  }
]
Found user: [
  {
    _id: new ObjectId('66c978099f2b2a56ee99af94'),
    name: 'Alice',
    age: 25
  },
  {
    _id: new ObjectId('66c97809f27fedd07292508b'),
    name: 'Bob',
    age: 30
  },
  {
    _id: new ObjectId('66c97809f27fedd07292508e'),
    name: 'Dave',
    age: 35
  },
  { _id: 1, name: 'Robbin', age: 26 },
  {
    _id: new ObjectId('66c97897ff14d6d2443dd297'),
    name: 'Robbin',
    age: 37
  },
  {
    _id: new ObjectId('66c97897ff14d6d2443dd298'),
    name: 'Alex',
 

In [7]:
// Count of all docs in 'users' collection [{}: no filter]
countDocs(collection = 'users', filter = {})
    .then(count => console.log("User count:", count))
    .catch(error => console.error("Error:", error));

Promise { <pending> }

Document count: 6
User count: 6


In [8]:
// Remove a doc
deleteDoc(collection = 'users', filter =  {_id: 1})
    .then((res) => {
        console.log(`Doc deleted: ${JSON.stringify(res)}`);
    })
    .catch((err) => {
        console.log("Doc deletion failed.");
    })

// Getting docs count
countDocs(collection = 'users', filter = {})
    .then(count => console.log("Active user count:", count))
    .catch(error => console.error("Error:", error));

Promise { <pending> }

Document deleted successfully: { acknowledged: true, deletedCount: 1 }
Doc deleted: {"acknowledged":true,"deletedCount":1}
Document count: 5
Active user count: 5


In [9]:
// Remove all docs
deleteDocs(collection = 'users', filter = {})
    .then((res) => {
        console.log(`Doc deleted: ${JSON.stringify(res)}`);
    })
    .catch((err) => {
        console.log("Doc deletion failed.");
    })

Promise { <pending> }

Documents deleted successfully: { acknowledged: true, deletedCount: 5 }
Doc deleted: {"acknowledged":true,"deletedCount":5}


## Simple filtering query examlpes
- Basic Equality
    ```
    filter = {
        category: "Electronics"
    }
    ```
  
- Accessing nested doc
   ```
   filter = {
        "role.type": "admin"
   }
   ```
- Comparison Operators
  ```
  filter = {
    price: { $gt: 500 }
  }
  
  filter = {
    price: { $lte: 1000 }
  }
  
  filter = {
    price: { $gte: 500, $lte: 1000 }
  }
  ```
  
- Logical Operators
  ```
  filter = {
    $and: [ { category: "Electronics" }, { inStock: true } ]
  }
  
  filter = {
    $or: [ { category: "Electronics" }, { price: { $lt: 500 } } ]
  }
  ```
  
- Filter for documents where a field value is in the given list:


  ```
  filter = { category: { $in: ["sale", "normal"] } }
  
  filter = { category: { $nin: ["sale", "normal"] } }
  ```

- Filter for documents where *price* field exists:

  ```
  filter = {
    price: { $exists: true }
  }

  filter = {
    price: { $exists: false }
  }
  ```

## Simple updating example
- To set a field value
  ```
  update = {
      $set: {age: 22}
  }
  ```
  
- To remove a field
  ```
  update = {
      $set: {age: ""}
  }
  ```
  
- To increment a field value
  ```
  update = {
      $inc: {age: 1}
  }
  ```
  
- To multiply a field value
  ```
  update = {
      $mult: {age: 2}
  }
  ```
  
- To rename a field
  ```
  update = {
      $rename: {oldname: "newname"}
  }
  ```
  
- To add and remove an item to an array
  ```
  update = {
      $push: {array: 6}
  }
  
  update = {
      $pull: {array: 6}
  }
  ```

## Nested documents

In [10]:
// Creating a doc and modifying its nested doc
createDoc(
    collection = 'users', 
    doc = {
      _id: new ObjectId(), // Generate a new ObjectId
      orderId: 12345,
      items: [
        { productId: "A001", quantity: 10 },
        { productId: "B002", quantity: 5 }
      ]
    })
    .then((res) => {
        console.log(`Doc created at ${JSON.stringify(res)}`);
        // After creating doc, we update its nested here immediately
        const productIdToUpdate = "B002";
        updateDoc(
            collection = 'users', 
            filter = {
                orderId: 12345,
                'items.productId': productIdToUpdate
            },
            update = {
                $set: { 'items.$.quantity': 10 } // '$' specify the first item mathces the filter
            })
            .then((res) => {
                console.log(`Doc updated: ${JSON.stringify(res)}`);
            })
            .catch((err) => {
                console.log("Doc update failed.");
            });
    })
    .catch((err) => {
        console.log("Doc creation failed.");
    });

// Creating a doc and modifying its nested doc with array filter
createDoc(
    collection = 'users', 
    doc = {
      _id: new ObjectId(), // Generate a new ObjectId
      orderId: 12345,
      items: [ // Nested docs
        { productId: "A001", quantity: 10 },
        { productId: "B002", quantity: 5 }
      ]
    })
    .then((res) => {
        console.log(`Doc created at ${JSON.stringify(res)}`);
        // After creating doc, we update its nested here immediately
        const productIdToUpdate = "B002";
        updateDoc(
            collection = 'users', 
            filter = { 
                orderId: 12345,
                // 'items.productId': productIdToUpdate
            },
            update = {
                $set: { 'items.$[item].quantity': 10 } // '$' specify the first item mathces the filter
            },
            options = {
              arrayFilters: [{ 'item.productId': productIdToUpdate }], // Array filter to match the element to update
              upsert: false // Do not create a new document if no matching document is found
            })
            .then((res) => {
                console.log(`Doc updated: ${JSON.stringify(res)}`);
            })
            .catch((err) => {
                console.log("Doc update failed.");
            });
    })
    .catch((err) => {
        console.log("Doc creation failed.");
    });

Promise { <pending> }

Document created successfully: new ObjectId('66c9789bff14d6d2443dd299')
Doc created at {"acknowledged":true,"insertedId":"66c9789bff14d6d2443dd299"}
Document created successfully: new ObjectId('66c9789bff14d6d2443dd29a')
Doc created at {"acknowledged":true,"insertedId":"66c9789bff14d6d2443dd29a"}
Document updated successfully: {
  acknowledged: true,
  modifiedCount: 1,
  upsertedId: null,
  upsertedCount: 0,
  matchedCount: 1
}
Doc updated: {"acknowledged":true,"modifiedCount":1,"upsertedId":null,"upsertedCount":0,"matchedCount":1}
Document updated successfully: {
  acknowledged: true,
  modifiedCount: 0,
  upsertedId: null,
  upsertedCount: 0,
  matchedCount: 1
}
Doc updated: {"acknowledged":true,"modifiedCount":0,"upsertedId":null,"upsertedCount":0,"matchedCount":1}


In [11]:
// Remove all docs
deleteDocs('users', {})
    .then((res) => {
        console.log(`Doc deleted: ${JSON.stringify(res)}`);
    })
    .catch((err) => {
        console.log("Doc deletion failed.");
    })

Promise { <pending> }

Documents deleted successfully: { acknowledged: true, deletedCount: 2 }
Doc deleted: {"acknowledged":true,"deletedCount":2}


## Transaction
MongoDB transactions are supported in the following scenarios:

    1- Replica Sets: Transactions require MongoDB to be set up as a replica set. Transactions ensure data consistency and durability across multiple nodes, which is a feature provided by replica sets.

    2 - Sharded Clusters: Transactions are also supported in sharded clusters, where transactions can span multiple shards.

**NOTE**: Following cell won't work unless one of the above requirements satisfied.

In [12]:
async function performTransaction(client) {
  const session = client.startSession(); // Starting a session
  try {
    session.startTransaction();
    
    const ordersCollection = client.db(dbName).collection('orders');
    const customersCollection = client.db(dbName).collection('customers');
    
    // Example operation: create a new customer
    await customersCollection.insertOne(
      { name: 'John Doe', email: 'john.doe@example.com' },
      { session }
    );

    // Example operation: create a new order
    await ordersCollection.insertOne(
      { orderNumber: 'ORD12345', amount: 100 },
      { session }
    );

    // Commit the transaction
    await session.commitTransaction();
    console.log('Transaction committed successfully');
  } catch (error) {
    // Abort the transaction if any operation fails
    await session.abortTransaction();
    console.error('Transaction aborted due to an error:', error);
  } finally {
    session.endSession();
  }
}
performTransaction(client)
    .then(() => { 
        console.log('Transaction finished.')
    })
    .catch((error) => console.error('Transaction failed:', error));


Promise { <pending> }

Transaction aborted due to an error: MongoServerError: Transaction numbers are only allowed on a replica set member or mongos
    at Connection.sendCommand (C:\Users\mehdi\Learning\JupyterLab\nodejs-learners-package\node_modules\mongodb\lib\cmap\connection.js:290:27)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async Connection.command (C:\Users\mehdi\Learning\JupyterLab\nodejs-learners-package\node_modules\mongodb\lib\cmap\connection.js:313:26)
    at async Server.command (C:\Users\mehdi\Learning\JupyterLab\nodejs-learners-package\node_modules\mongodb\lib\sdam\server.js:167:29)
    at async InsertOneOperation.executeCommand (C:\Users\mehdi\Learning\JupyterLab\nodejs-learners-package\node_modules\mongodb\lib\operations\command.js:73:16)
    at async InsertOneOperation.execute (C:\Users\mehdi\Learning\JupyterLab\nodejs-learners-package\node_modules\mongodb\lib\operations\insert.js:37:16)
    at async InsertOneOperation.execute (C:\Users\mehdi\

Transaction finished.


## Bulk write

In [16]:
async function runBulkWrite() {

  try {
    const db = client.db(dbName);
    const collection = db.collection('users');

    // Define bulkWrite operations
    const operations = [
      { insertOne: { document: { name: 'Alice', age: 25 } } },
      { updateOne: { filter: { name: 'Alice' }, update: { $set: { age: 30 } }, upsert: true } },
      { deleteOne: { filter: { name: 'Charlie' } } },
      { replaceOne: { filter: { name: 'Dave' }, replacement: { name: 'Dave', age: 35 }, upsert: true } }
    ];

    // Execute bulkWrite operation
    const result = await collection.bulkWrite(operations);
    console.log('Bulk write result:', result);
  } finally {
  }
}

runBulkWrite().then(() => {
}).catch(console.error);


Promise { <pending> }

Bulk write result: BulkWriteResult {
  insertedCount: 1,
  matchedCount: 2,
  modifiedCount: 1,
  deletedCount: 0,
  upsertedCount: 0,
  upsertedIds: {},
  insertedIds: { '0': new ObjectId('66c97bfcff14d6d2443dd29f') }
}


In [17]:
 // Remove all docs
deleteDocs('users', {})
    .then((res) => {
        console.log(`Doc deleted: ${JSON.stringify(res)}`);
    })
    .catch((err) => {
        console.log("Doc deletion failed.");
    });

Promise { <pending> }

Documents deleted successfully: { acknowledged: true, deletedCount: 5 }
Doc deleted: {"acknowledged":true,"deletedCount":5}


## Closing the Database Connection

In [12]:
client.close()
    .then(() =>{ 
        console.log('MongoDB connection closed');
    })
    .catch((error) => { 
        console.error('Error closing the MongoDB connection:', error);
    })

Promise { <pending> }

MongoDB connection closed
